## Import

In [1]:
from catboost import CatBoostClassifier, CatBoostRegressor
from xgboost import XGBClassifier, XGBRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from ngboost import NGBClassifier, NGBRegressor
from collections import Counter
from tqdm import tqdm
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings(action='default')

## Load

In [3]:
out_data_path = 'C:\\Users\\PC0\\Documents\\datasets\\stock_price\\'
data_path = 'C:\\Users\\PC0\\Documents\\GitHub\\Stock-price-prediction\\datasets\\'

C:\Users\PC0\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [4]:
stock_df = pd.read_csv(data_path + "stock_df_01.csv")

In [5]:
n_target = np.array(list(stock_df.target))
c_target = np.array(list(stock_df.c_target))
n_target2 = n_target - np.array(stock_df['kospi-200_close'])

In [6]:
del stock_df['target']
del stock_df['c_target']

In [7]:
stock_df.shape

(2410, 372)

In [8]:
var_cols = [x for x in stock_df.columns if "var" in x]

In [9]:
stock_df.head()

,date,kosdaq_close,kosdaq_vol,kosdaq_var,us-30_close,us-30_vol,us-30_var,us-spx-500_close,us-spx-500_var,nasdaq-composite_close,nasdaq-composite_vol,nasdaq-composite_var,smallcap-2000_close,smallcap-2000_var,volatility-s-p-500_close,volatility-s-p-500_var,s-p-tsx-composite_close,s-p-tsx-composite_vol,s-p-tsx-composite_var,bovespa_close,bovespa_vol,bovespa_var,ipc_close,ipc_vol,ipc_var,germany-30_close,germany-30_vol,germany-30_var,uk-100_close,uk-100_vol,uk-100_var,france-40_close,france-40_vol,france-40_var,eu-stoxx50_close,eu-stoxx50_vol,eu-stoxx50_var,netherlands-25_close,netherlands-25_vol,netherlands-25_var,spain-35_close,spain-35_vol,spain-35_var,it-mib-40_close,it-mib-40_vol,it-mib-40_var,switzerland-20_close,switzerland-20_vol,switzerland-20_var,psi-20_close,psi-20_vol,psi-20_var,bel-20_close,bel-20_vol,bel-20_var,omx-stockholm-30_close,omx-stockholm-30_vol,omx-stockholm-30_var,mcx_close,mcx_var,...,가솔린_RBOB_close_x.2,가솔린_RBOB_vol_x.2,가솔린_RBOB_var_x.2,난방유_close_y.1,난방유_vol_y.1,난방유_var_y.1,런던_가스_오일_close_x.1,런던_가스_오일_vol_x.1,런던_가스_오일_var_x.1,브렌트유_close_y,브렌트유_vol_y,브렌트유_var_y,천연가스_close_x,천연가스_vol_x,천연가스_var_x,WTI유_close,WTI유_vol,WTI유_var,가솔린_RBOB_close_y.2,가솔린_RBOB_vol_y.2,가솔린_RBOB_var_y.2,난방유_close,난방유_vol,난방유_var,런던_가스_오일_close_y.1,런던_가스_오일_vol_y.1,런던_가스_오일_var_y.1,브렌트유_close,브렌트유_vol,브렌트유_var,천연가스_close_y,천연가스_vol_y,천연가스_var_y,탄소배출권_close,탄소배출권_vol,탄소배출권_var,미국_구리_close,미국_구리_vol,미국_구리_var,영국_구리_close,영국_구리_var,금_close,금_vol,금_var,영국_납_close,영국_납_var,영국_니켈_close,영국_니켈_var,미국_백금_close,미국_백금_var,영국_아연_close,영국_아연_vol,영국_아연_var,은_close,은_vol,은_var,영국_주석_close,영국_주석_var,미국_팔라듐_close,미국_팔라듐_var
0,2012-01-03,513.83,590990.0,1.39,12397.38,152560000.0,1.47,1277.06,1.55,2648.72,411790000.0,1.67,752.26,1.53,22.97,-1.84,12208.43,162330000.0,2.12,59265.0,3080000.0,2.48,37384.34,140640000.0,0.13,6166.57,116710000.0,1.50,5699.91,778530000.0,2.29,3245.40,123420000.0,0.72,2389.91,58260000.0,0.83,318.04,84080000.0,0.39,8732.4,186690000.0,0.10,15645.56,486170000.0,1.24,6050.93,48740000.0,1.93,5700.94,152880000.0,1.60,2146.45,16920000.0,1.52,1018.8000,99030000.0,1.9400,1444.76,3.03,...,2.7486,43590.0,2.32,3.0382,62570.0,3.43,953.50,51390.0,2.90,112.13,178620.0,4.42,2.993,106190.0,0.13,102.96,275480.0,4.18,2.7486,43590.0,2.32,3.0382,62570.0,3.43,953.50,51390.0,2.90,112.13,178620.0,4.42,2.993,106190.0,0.13,6.28,310.0,-12.11,3.525,250.0,2.71,7782.25,2.47,1599.7,230.0,2.17,2081.50,3.16,18917.0,0.99,1436.20,1.96,1864.25,9410.0,1.83,29.533,140.0,4.88,19949.0,4.13,664.8,1.42
1,2012-01-04,516.30,652090.0,0.48,12418.42,145130000.0,0.17,1277.30,0.02,2648.36,411090000.0,-0.01,747.28,-0.66,22.22,-3.27,12226.47,175580000.0,0.15,59365.0,2250000.0,0.17,37387.63,150440000.0,0.01,6111.55,100870000.0,-0.89,5668.45,696940000.0,-0.55,3193.65,114040000.0,-1.59,2349.89,62960000.0,-1.67,314.53,77440000.0,-1.10,8581.8,243800000.0,-1.72,15327.03,466040000.0,-2.04,6058.08,41440000.0,0.12,5592.76,176190000.0,-1.90,2121.11,14010000.0,-1.18,1006.3100,112810000.0,-1.2300,1447.59,0.20,...,2.7852,52340.0,1.33,3.0899,75090.0,1.70,964.75,70870.0,1.18,113.70,221270.0,1.40,3.096,133510.0,3.44,103.22,271710.0,0.25,2.7852,52340.0,1.33,3.0899,75090.0,1.70,964.75,70870.0,1.18,113.70,221270.0,1.40,3.096,133510.0,3.44,6.27,2200.0,-0.16,3.429,150.0,-2.72,7531.25,-3.23,1611.9,170.0,0.76,2042.50,-1.87,18810.0,-0.57,1426.05,-0.71,1856.50,8430.0,-0.42,29.063,70.0,-1.59,19555.0,-1.98,649.9,-2.24
2,2012-01-05,521.96,654300.0,1.10,12415.70,158440000.0,-0.02,1281.06,0.29,2669.86,442080000.0,0.81,752.29,0.67,21.48,-3.33,12237.40,161560000.0,0.09,58546.0,2350000.0,-1.38,37017.95,187660000.0,-0.99,6095.99,141740000.0,-0.25,5624.26,794280000.0,-0.78,3144.91,121160000.0,-1.53,2315.75,74720000.0,-1.45,311.92,87480000.0,-0.83,8329.6,192700000.0,-2.94,14767.22,682930000.0,-3.65,6026.57,46880000.0,-0.52,5505.65,136880000.0,-1.56,2096.90,15340000.0,-1.14,1002.1700,66480000.0,-0.4100,1434.91,-0.88,...,2.7365,48710.0,-1.75,3.0388,80940.0,-1.65,966.50,48360.0,0.18,112.74,205410.0,-0.84,2.980,16314

## Modeling

In [36]:
# temp_x = stock_df.loc[:, var_cols]
temp_x = stock_df.iloc[:, 1:]

* baseline modeling - stacked sample

In [37]:
from sklearn.metrics.pairwise import cosine_similarity

In [43]:
train_volum = 100
train_start_idx = 0
sample_days = 5

data = temp_x

model_xgb = XGBRegressor()
model_lgbm = LGBMRegressor()
model_ngb = NGBRegressor()
model_cat = CatBoostRegressor(verbose = False)

model = model_xgb

In [44]:
c_target_num = pd.DataFrame(n_target2)

In [45]:
pred_dict = {}

In [46]:
beta = 0.999

In [ ]:
train_end_idx = train_start_idx + train_volum
test_start_idx = train_end_idx + 1

whole_predict = []
whole_truth = []
whole_acc = []

print("Start Fitting..")
for _ in tqdm(range(data.shape[0] - (train_volum + 1))) :
    temp_train_x = data.loc[:train_end_idx,:]
    temp_train_y = c_target_num.loc[:train_end_idx]
    temp_test_x = data.loc[test_start_idx:test_start_idx,:]
    temp_test_y = c_target_num.loc[test_start_idx]

    # append sample
    temp_train_x = temp_train_x.append(temp_train_x.loc[(train_end_idx - sample_days):])
    temp_train_y = temp_train_y.append(temp_train_y.loc[(train_end_idx - sample_days):])
    
    #sim_mat = cosine_similarity(np.array(temp_train_x))
    
    sample_w = [beta ** x for x in range((temp_train_x.shape[0]), 0, -1)]
    sample_w = np.array(sample_w) + 1-np.mean(sample_w)
    
    # fitting
    model.fit(temp_train_x,temp_train_y, sample_weight=sample_w)
    #model.fit(temp_train_x,temp_train_y, sample_weight=sim_mat[-1])
    #model.fit(temp_train_x,temp_train_y)
    
    # predict
    predict_value = model.predict(temp_test_x)

    # save acc
    pred_dict[stock_df.date[test_start_idx]] = predict_value[0]
    whole_predict.append(predict_value[0])
    whole_truth.append(temp_test_y.iloc[0])

    train_start_idx += 1
    train_end_idx = train_start_idx + train_volum
    test_start_idx = train_end_idx + 1

print("Finish")

  0%|                                                                                         | 0/2309 [00:00<?, ?it/s]

Start Fitting..


 30%|███████████████████████▌                                                       | 690/2309 [06:01<20:27,  1.32it/s]

* lightgbm performance - with sample weights 0.999 beta adjust

In [21]:
mean_absolute_error(whole_truth, whole_predict)

C:\Users\PC0\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


2.1128765783408823

In [22]:
r2_score(whole_truth, whole_predict)

C:\Users\PC0\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


-0.017451403212568106

* xgboost performance - with sample weights 0.999 beta adjust

In [ ]:
mean_absolute_error(whole_truth, whole_predict)

In [ ]:
r2_score(whole_truth, whole_predict)

In [ ]:
whole_truth[-10:]

In [ ]:
whole_predict[-10:]